In [1]:
import os
os.environ['ESMFMKFILE'] = '/home/malasawedah/.conda/envs/test2/lib/esmf.mk'

# Now you can import the ESMF library
#import ESMF


In [2]:
import datetime
import geopandas as gpd
import pandas as pd
import xarray as xr
import numpy as np
#import xvec
import shapely
from shapely import Point, box
import esmpy
import xagg as xa
import rioxarray

In [73]:
aoi = gpd.read_file('/mnt/CEPH_PROJECTS/ScaleAgData/03_Ancillary_Data/LAFIS_zonal_stats_prato_stabile_1000.shp')
grid5 = pd.read_csv('/mnt/CEPH_PROJECTS/ScaleAgData/02_EO_Data/s1/polygons_grid_5')
s1 = xr.open_dataset(f'/mnt/CEPH_PROJECTS/ScaleAgData/02_EO_Data/s1/latest/GRD_luca/asc/grid5/S1_grid5_1/openEO.nc')

In [74]:
s1

<xarray.Dataset>
Dimensions:  (t: 12, x: 3764, y: 2282)
Coordinates:
  * t        (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-02 2021-04-03
  * x        (x) float64 6.478e+05 6.478e+05 6.478e+05 ... 6.854e+05 6.854e+05
  * y        (y) float64 5.17e+06 5.17e+06 5.17e+06 ... 5.147e+06 5.147e+06
Data variables:
    crs      |S1 ...
    VV       (t, y, x) float32 ...
    VH       (t, y, x) float32 ...
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform - Geotrellis backend: 0.15.1a1
    description:  
    title:

In [75]:
s1.rio.write_crs("EPSG:32632", inplace=True)

<xarray.Dataset>
Dimensions:  (t: 12, x: 3764, y: 2282)
Coordinates:
  * t        (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-02 2021-04-03
  * x        (x) float64 6.478e+05 6.478e+05 6.478e+05 ... 6.854e+05 6.854e+05
  * y        (y) float64 5.17e+06 5.17e+06 5.17e+06 ... 5.147e+06 5.147e+06
    crs      int64 0
Data variables:
    VV       (t, y, x) float32 ...
    VH       (t, y, x) float32 ...
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform - Geotrellis backend: 0.15.1a1
    description:  
    title:

In [76]:
s1

<xarray.Dataset>
Dimensions:  (t: 12, x: 3764, y: 2282)
Coordinates:
  * t        (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-02 2021-04-03
  * x        (x) float64 6.478e+05 6.478e+05 6.478e+05 ... 6.854e+05 6.854e+05
  * y        (y) float64 5.17e+06 5.17e+06 5.17e+06 ... 5.147e+06 5.147e+06
    crs      int64 0
Data variables:
    VV       (t, y, x) float32 ...
    VH       (t, y, x) float32 ...
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform - Geotrellis backend: 0.15.1a1
    description:  
    title:

In [77]:
s1_wgs84 = s1.rio.reproject("EPSG:4326")

In [78]:
s1_wgs84

<xarray.Dataset>
Dimensions:  (x: 4159, y: 1788, t: 12)
Coordinates:
  * x        (x) float64 10.92 10.92 10.92 10.92 ... 11.42 11.42 11.42 11.42
  * y        (y) float64 46.67 46.67 46.67 46.67 ... 46.46 46.46 46.46 46.46
  * t        (t) datetime64[ns] 2021-03-03 2021-03-04 ... 2021-04-02 2021-04-03
    crs      int64 0
Data variables:
    VV       (t, y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
    VH       (t, y, x) float32 nan nan nan nan nan nan ... nan nan nan nan nan
Attributes:
    Conventions:  CF-1.9
    institution:  openEO platform - Geotrellis backend: 0.15.1a1
    description:  
    title:

In [79]:
grid5['geometry'] = grid5['geometry'].apply(lambda x: shapely.wkt.loads(x))
gdf_grid5 = gpd.GeoDataFrame(grid5, geometry='geometry')
gdf_grid5.crs = 'EPSG:4326'
gdf_grid5.head(3)

,Unnamed: 0,ID,FID,CUAA,CODE,DESCR_IT,DESCR_DE,area,dem_mean,dem_median,slo_mean,slo_median,asp_mean,asp_median,north_mean,north_medi,dem_class,geometry
0,0,2894152,PRGMFR75E13F132Z-1688030,PRGMFR75E13F132Z,AP2,Prato stabile,Wiese (Dauerwiese),56,1301.867221,1301.705200,17.897289,17.806307,85.131834,84.665092,1.000000,1.0,1300.0,"POLYGON ((11.02662 46.46839, 11.02669 46.46830..."
1,1,2347633,RNRGTH80H18F132V-59,RNRGTH80H18F132V,AP2,Prato stabile,Wiese (Dauerwiese),31147,1302.440111,1302.553589,22.939459,22.398680,145.304115,153.931000,0.043887,0.0,1300.0,"POLYGON ((10.94554 46.65611, 10.94556 46.65604..."
2,2,2970168,WSSCRS85L19A952F-2158250,WSSCRS85L19A952F,AP2,Prato stabile,Wiese (Dauerwiese),842,1318.083754,1317.940063,12.269650,12.262682,228.961556,228.782425,0.000000,0.0,1300.0,"POLYGON ((11.13220 46.49439, 11.13249 46.49438..."


In [83]:
weightmap = xa.pixel_overlaps(s1_wgs84,gdf_grid5)

creating polygons for each pixel...
lat/lon bounds not found in dataset; they will be created.
calculating overlaps between pixels and output polygons...
success!


In [ ]:
# Aggregate
aggregated = xa.aggregate(s1_wgs84,weightmap)

adjusting grid... (this may happen because only a subset of pixels were used for aggregation for efficiency - i.e. [subset_bbox=True] in xa.pixel_overlaps())


In [ ]:
ds_out = aggregated.to_dataset()
ds_out

In [ ]:
ds_out_32632 = ds_out.rio.reproject("EPSG:32632")

In [ ]:
ds_out_32632